In [1]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision
from nltk.translate.chrf_score import sentence_chrf, corpus_chrf
from nltk.metrics import scores
import scipy.io.wavfile
from IPython.display import Audio
from IPython.display import display
from nltk.stem import *
# from nltk.stem.snowball import SnowballStemmer
from stemming.porter2 import stem
import stemming
from nltk.metrics.scores import recall

from nltk.corpus import stopwords

%matplotlib inline

In [2]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)

In [3]:
smooth_fun = nltk.translate.bleu_score.SmoothingFunction()

In [4]:
from bow_run import *

In [5]:
# def count_match(list1, list2):
#     # each list can have repeated elements. The count should account for this.
#     count1 = Counter(list1)
#     count2 = Counter(list2)
#     count2_keys = count2.keys()
#     common_w = set(count1.keys()) & set(count2_keys)
#     print(common_w)
#     matches = sum([min(count1[w], count2[w]) for w in common_w])
#     return matches

In [6]:
# def get_bow_batch(m_dict, x_key, y_key, utt_list, vocab_dict, bow_dict,
#                   max_enc, max_dec, input_path=''):
#     batch_data = {'X':[], 't':[], 'y':[], 'r':[]}
#     # -------------------------------------------------------------------------
#     # loop through each utterance in utt list
#     # -------------------------------------------------------------------------
#     for i, u in enumerate(utt_list):
#         # ---------------------------------------------------------------------
#         #  add X data
#         # ---------------------------------------------------------------------
#         if x_key == 'sp':
#             # -----------------------------------------------------------------
#             # for speech data
#             # -----------------------------------------------------------------
#             # get path to speech file
#             utt_sp_path = os.path.join(input_path, "{0:s}.npy".format(u))
#             if not os.path.exists(utt_sp_path):
#                 # for training data, there are sub-folders
#                 utt_sp_path = os.path.join(input_path,
#                                            u.split('_',1)[0],
#                                            "{0:s}.npy".format(u))
#             if os.path.exists(utt_sp_path):
#                 x_data = xp.load(utt_sp_path)[:max_enc]
#             else:
#                 # -------------------------------------------------------------
#                 # exception if file not found
#                 # -------------------------------------------------------------
#                 raise FileNotFoundError("ERROR!! file not found: {0:s}".format(utt_sp_path))
#                 # -------------------------------------------------------------
#         else:
#             # -----------------------------------------------------------------
#             # for text data
#             # -----------------------------------------------------------------
#             x_ids = [vocab_dict[x_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][x_key]]
#             x_data = xp.asarray(x_ids, dtype=xp.int32)[:max_enc]
#             # -----------------------------------------------------------------
#         # ---------------------------------------------------------------------
#         #  add labels
#         # ---------------------------------------------------------------------
#         if type(m_dict[u][y_key]) == list:
#             en_ids = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key]])-set(range(4)))
#             r_data = [en_ids[:max_dec]]

#         else:
#             # dev and test data have multiple translations
#             # choose the first one for computing perplexity
#             en_ids = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key][0]])-set(range(4)))
#             r_data = []
#             for r in m_dict[u][y_key]:
#                 r_list = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in r])-set(range(4)))
#                 r_data.append(r_list[:max_dec])

#         y_ids = en_ids[:max_dec]
#         # ---------------------------------------------------------------------
#         if len(x_data) > 0:
#             #  and len(y_ids) > 0
#             batch_data['X'].append(x_data)
#             batch_data['t'].append([y_ids])
#             y_data = xp.zeros(len(bow_dict['w2i']), dtype=xp.int32)
#             y_data[y_ids] = 1
#             y_data[list(range(4))] = -1
#             batch_data['y'].append(y_data)
#             batch_data['r'].append(r_data)

#     # -------------------------------------------------------------------------
#     # end for all utterances in batch
#     # -------------------------------------------------------------------------
#     if len(batch_data['X']) > 0 and len(batch_data['y']) > 0:
#         batch_data['X'] = F.pad_sequence(batch_data['X'], padding=PAD_ID)
#         batch_data['y'] = F.pad_sequence(batch_data['y'], padding=PAD_ID)
#     return batch_data

In [7]:
# def basic_precision_recall(r, h, display=False):
#     p_numerators = Counter() # Key = ngram order, and value = no. of ngram matches.
#     p_denominators = Counter() # Key = ngram order, and value = no. of ngram in ref.
#     r_numerators = Counter() # Key = ngram order, and value = no. of ngram matches.
#     r_denominators = Counter() # Key = ngram order, and value = no. of ngram in ref.

#     print("total utts={0:d}".format(len(r)))

#     i=1

#     for references, hypothesis in zip(r, h):
#         p_i = modified_precision(references, hypothesis, i)
#         p_numerators[i] += p_i.numerator
#         p_denominators[i] += p_i.denominator

#         tot_match = 0
#         tot_count = 0
        
#         max_recall_match = 0
#         max_recall_count = 0
#         max_recall = 0
        
#         for curr_ref in references:
#             print(curr_ref, hypothesis)
#             curr_match = count_match(curr_ref, hypothesis)
#             print("curr_match", curr_match)
            
#             curr_count = len(curr_ref)
#             curr_recall = curr_match / curr_count if curr_count > 0 else 0
#             print(curr_match, curr_count, curr_recall)
#             if curr_recall > max_recall:
#                 max_recall_match = curr_match
#                 max_recall_count = curr_count
#                 max_recall = curr_recall
                
#         r_numerators[i] += max_recall_match
#         r_denominators[i] += max_recall_count

#     prec = [(n / d) * 100 if d > 0 else 0 for n,d in zip(p_numerators.values(), p_denominators.values())]
#     rec = [(n / d) * 100 if d > 0 else 0 for n,d in zip(r_numerators.values(), r_denominators.values())]

#     if display:
#         print("{0:10s} | {1:>8s}".format("metric", "1-gram"))
#         print("-"*54)
#         print("{0:10s} | {1:8.2f}".format("precision", *prec))
#         print("{0:10s} | {1:8.2f}".format("recall", *rec))

#     return prec[0], rec[0]

In [10]:
cfg_path = "./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5"

In [11]:
last_epoch, model, optimizer, m_cfg, t_cfg = check_model(cfg_path)

cnn_out_dim = rnn_in_units =  640


/home/sameer/anaconda2/envs/chainer3/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.links.normalization.layer_normalization.py is experimental. The interface can change in the future.
  FutureWarning)


using SGD optimizer
--------------------------------------------------------------------------------
model found = 
./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5/seq2seq_35.model
finished loading ..
optimizer found = ./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5/train.opt
finished loading optimizer ...


In [12]:
modified_precision([[0],[0]], [1], n=1)

Fraction(0, 1)

In [13]:
basic_precision_recall([[[2],[1]]], [[2]])

total utts=1


(100.0, 100.0)

In [14]:
m_cfg['model_dir']

'./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5'

### Fisher dev

In [15]:
%%capture
train_key = m_cfg['train_set']
dev_key = m_cfg['dev_set']
batch_size=t_cfg['batch_size']
enc_key=m_cfg['enc_key']
dec_key=m_cfg['dec_key']
input_path = os.path.join(m_cfg['data_path'], m_cfg['dev_set'])
# -------------------------------------------------------------------------
# get data dictionaries
# -------------------------------------------------------------------------
map_dict, vocab_dict, bucket_dict, bow_dict = get_data_dicts(m_cfg)
if os.path.exists(os.path.join(m_cfg['model_dir'], "model_s2t_dev_out.dict")):
    dev_utts = pickle.load(open(os.path.join(m_cfg['model_dir'], "model_s2t_dev_out.dict"), "rb"))

if os.path.exists(os.path.join(m_cfg['model_dir'], "mean_pos_scores.dict")):
    mean_pos_scores = pickle.load(open(os.path.join(m_cfg['model_dir'], "mean_pos_scores.dict"), "rb"))
    mean_neg_scores = pickle.load(open(os.path.join(m_cfg['model_dir'], "mean_neg_scores.dict"), "rb"))
# batch_size = {'max': 128, 'med': 128, 'min': 128, 'scale': 1}
batch_size = {'max': 64, 'med': 64, 'min': 64, 'scale': 1}

In [16]:
random.seed("meh")
# random.seed("haha")

In [17]:
optimizer.hyperparam.lr = 1.0

In [23]:
xp.vstack((mean_pos_scores, mean_neg_scores))[:,4:10]

array([[ 0.34183857,  0.20332989,  0.16533113,  0.13396151,  0.12785606,
         0.17088211],
       [ 0.14858542,  0.08471115,  0.07898171,  0.07449739,  0.05836316,
         0.06727904]], dtype=float32)

In [25]:
xp.mean(mean_pos_scores), xp.mean(mean_neg_scores)

(array(0.04837069660425186, dtype=float32),
 array(0.019010351970791817, dtype=float32))

In [26]:
xp.std(mean_pos_scores), xp.std(mean_neg_scores)

(array(0.05061212554574013, dtype=float32),
 array(0.020977046340703964, dtype=float32))

In [53]:
for i in range(5):
    train_utts, train_loss = feed_model(model,
                                optimizer=optimizer,
                                m_dict=map_dict[m_cfg['train_set']],
                                b_dict=bucket_dict[m_cfg['train_set']],
                                vocab_dict=vocab_dict,
                                bow_dict=bow_dict,
                                batch_size=batch_size,
                                x_key=enc_key,
                                y_key=dec_key,
                                train=True,
                                input_path=os.path.join(m_cfg['data_path'], m_cfg['train_set']),
                                max_dec=m_cfg['max_en_pred'],
                                t_cfg=t_cfg,
                                use_y=True)
    print(train_loss)
    print("using threshold of 0.1")
    train_pred_words = get_pred_words_from_probs(train_utts["probs"],
                                           0.1,
                                           m_cfg['max_en_pred'])
    print(basic_precision_recall(train_utts["refs"], train_pred_words))
    print("using mean positive prediction threshold")
    train_pred_words = get_pred_words_from_probs(train_utts["probs"],
                                               mean_pos_scores,
                                               m_cfg['max_en_pred'])
    print(basic_precision_recall(train_utts["refs"], train_pred_words))

b=30,l=0.09,avg=0.09: 100%|█████████████████| 1305/1305 [01:48<00:00,  2.92it/s]


0.08796076363981215
using threshold of 0.1
total utts=1305
(19.01918976545842, 56.74300254452926)
using mean positive prediction threshold


  0%|                                                  | 0/1305 [00:00<?, ?it/s]

total utts=1305
(8.627168785480132, 65.47330097087378)


b=116,l=0.09,avg=0.09: 100%|████████████████| 1305/1305 [01:49<00:00,  3.06it/s]


...

0.08465433786553678
using threshold of 0.1
total utts=1305
(19.953742640874687, 56.42092746730083)
using mean positive prediction threshold


  0%|                                                  | 0/1305 [00:00<?, ?it/s]

total utts=1305
(9.127631476734251, 67.80150028851702)


b=61,l=0.10,avg=0.08: 100%|█████████████████| 1305/1305 [01:48<00:00,  7.91it/s]


0.08406977717583972
using threshold of 0.1
total utts=1305
(19.453781512605044, 59.28297055057619)
using mean positive prediction threshold
total utts=1305
(8.573497465604635, 68.32083092902481)


In [29]:
# model_fil = m_cfg['model_fname']
# print("Saving model")
# serializers.save_npz(model_fil.replace(".model", "_{0:d}.model".format(20)), model)
# print("Finished saving model")
# print("Saving optimizer")
# serializers.save_npz(m_cfg['opt_fname'], optimizer)
# print("Finished saving optimizer")
# print("Saving utterance predictions")

# pickle.dump(utts, open(os.path.join(m_cfg['model_dir'], "model_s2t_dev_out.dict"), "wb"))
# pickle.dump(mean_pos_scores, open(os.path.join(m_cfg['model_dir'], "mean_pos_scores.dict"), "wb"))
# pickle.dump(mean_neg_scores, open(os.path.join(m_cfg['model_dir'], "mean_neg_scores.dict"), "wb"))

In [37]:
train_utts["probs"][:2,4]

array([-2.59993172, -2.21785092], dtype=float32)

In [38]:
train_utts["probs"][:2,:10], train_utts["refs"][:2][:10]

(array([[ 0.06181449, -0.40225312,  0.15357459,  0.31289765, -2.59993172,
         -1.86838281, -1.92051077, -1.57338321, -2.54046893, -3.31746721],
        [ 0.08252125, -0.15521246, -0.17448367, -0.22369032, -2.21785092,
         -2.01475859, -1.14290965, -2.17804217, -3.90053344, -5.28072119]], dtype=float32),
 [[[66, 51, 85, 55]], [[41, 27, 23]]])

In [39]:
mean_pos_scores = xp.array([0.0 for i in bow_dict["i2w"]], dtype="f")
mean_neg_scores = xp.array([0.0 for i in bow_dict["i2w"]], dtype="f")

In [40]:
for i in range(4, len(bow_dict["i2w"])):
    this_word = bow_dict["i2w"][i]
    print(this_word, bow_dict["freq"][this_word])
    pos_indx = [i in r[0] for r in train_utts["refs"]]
    neg_indx = [i not in r[0] for r in train_utts["refs"]]
    mean_pos_scores[i] = np.mean(F.sigmoid(train_utts["probs"][:,i][pos_indx]).data)
    mean_neg_scores[i] = np.mean(F.sigmoid(train_utts["probs"][:,i][neg_indx]).data)
    print(mean_pos_scores[i], train_utts["probs"][:,i][pos_indx].shape)
    print(mean_neg_scores[i], train_utts["probs"][:,i][neg_indx].shape)

b'yes' 35054
0.32988789677619934 (251,)
0.12474673986434937 (1054,)
b'like' 14334
0.2018955796957016 (98,)
0.08374379575252533 (1207,)
b'well' 12354
0.1673634946346283 (90,)
0.08193064481019974 (1215,)
b'know' 10619
0.14862149953842163 (89,)
0.08002760261297226 (1216,)
b'ah' 9590
0.11550240218639374 (82,)
0.054315224289894104 (1223,)
b'oh' 9183
0.14873015880584717 (91,)
0.05575650930404663 (1214,)
b'eh' 7164
0.13792714476585388 (62,)
0.048028115183115005 (1243,)
b'think' 6677
0.10452914983034134 (49,)
0.03986036032438278 (1256,)
b'people' 6560
0.14330370724201202 (60,)
0.05451585352420807 (1245,)
b'right' 6505
0.043389104306697845 (45,)
0.031220421195030212 (1260,)
b'one' 6340
0.10720238834619522 (62,)
0.042309608310461044 (1243,)
b'aha' 6291
0.13751831650733948 (60,)
0.03354032710194588 (1245,)
b'uh' 5595
0.05088953301310539 (35,)
0.029370009899139404 (1270,)
b'sure' 4842
0.07231127470731735 (48,)
0.033011987805366516 (1257,)
b'say' 4652
0.13842333853244781 (44,)
0.0487004779279232 (1

In [41]:
mean_neg_scores, mean_pos_scores

(array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.12474674,
         0.0837438 ,  0.08193064,  0.0800276 ,  0.05431522,  0.05575651,
         0.04802812,  0.03986036,  0.05451585,  0.03122042,  0.04230961,
         0.03354033,  0.02937001,  0.03301199,  0.04870048,  0.0221343 ,
         0.04068871,  0.03272517,  0.01477878,  0.02946863,  0.02518984,
         0.03295006,  0.02366619,  0.03679598,  0.0280505 ,  0.02612888,
         0.02280403,  0.02703858,  0.0232062 ,  0.02811868,  0.02252786,
         0.01181588,  0.02264206,  0.02524832,  0.00821574,  0.00830746,
         0.01029135,  0.01741837,  0.016679  ,  0.00896313,  0.01475668,
         0.00748555,  0.01278598,  0.01479203,  0.00731342,  0.01057485,
         0.00854175,  0.01324674,  0.01264612,  0.01502661,  0.0141947 ,
         0.01338058,  0.02119661,  0.02020238,  0.00361494,  0.01325324,
         0.00719902,  0.00933194,  0.01690347,  0.00850862,  0.01268221,
         0.01296028,  0.01619619,  0.00584027,  0.0

In [45]:
check_index = 4
this_word = bow_dict["i2w"][check_index]
print(this_word, bow_dict["freq"][this_word])
pos_indx = [check_index in r[0] for r in train_utts["refs"]]
neg_indx = [check_index not in r[0] for r in train_utts["refs"]]
print(np.mean(F.sigmoid(train_utts["probs"][:,check_index][pos_indx]).data), train_utts["probs"][:,check_index][pos_indx].shape)
print(np.mean(F.sigmoid(train_utts["probs"][:,check_index][neg_indx]).data), train_utts["probs"][:,check_index][neg_indx].shape)

b'yes' 35054
0.32988789677619934 (251,)
0.12474673986434937 (1054,)


In [46]:
xp.all(train_utts["probs"][:10] == train_utts["probs"][10:20]), train_utts["preds"][:10] == train_utts["preds"][10:20]

(array(False, dtype=bool), False)

In [47]:
train_prec, train_rec = basic_precision_recall(train_utts["refs"], train_utts["preds"])
print(train_prec, train_rec)

total utts=1305
4.954268292682927 58.03571428571429


In [51]:
PRED_THRESH = 0.1
pred_words = []
pred_limit = m_cfg['max_en_pred']
for row in F.sigmoid(train_utts["probs"]).data:
    pred_inds = xp.where(row >=PRED_THRESH)[0]
    if len(pred_inds) > pred_limit:
        pred_inds = xp.argsort(row)[-pred_limit:][::-1]
    #pred_words.append([bow_dict['i2w'][i] for i in pred_inds.tolist()])
    pred_words.append([i for i in pred_inds.tolist() if i > 3])
    print(row)
    print(mean_pos_scores)
    print(pred_inds)
    break

train_prec, train_rec = basic_precision_recall(train_utts["refs"], pred_words)
print(train_prec, train_rec)

[ 0.51544869  0.40077111  0.53831834  0.57759237  0.06914282  0.13372895
  0.12780462  0.17173462  0.07306941  0.0349768   0.11242215  0.03428413
  0.19470397  0.01632826  0.07281633  0.00395539  0.0192567   0.01851128
  0.07345533  0.00187328  0.02366112  0.02286559  0.03503358  0.0583662
  0.02522211  0.08731006  0.03314728  0.15134153  0.05322033  0.08109252
  0.05765366  0.10448772  0.03016962  0.03833305  0.05536424  0.02219497
  0.04001034  0.08084626  0.00168835  0.00555753  0.01220204  0.03535946
  0.04005387  0.01693087  0.02857115  0.01161144  0.02318759  0.00666831
  0.01684841  0.01795754  0.01862131  0.04898268  0.0231012   0.01412547
  0.04483757  0.03984403  0.02511185  0.02505676  0.000625    0.0297839
  0.00900214  0.01389109  0.03104129  0.008212    0.02086379  0.01783746
  0.04733364  0.01644694  0.03997594  0.03147162  0.03201365  0.00521437
  0.01047211  0.00311684  0.00436865  0.01912993  0.0278133   0.02110873
  0.01631512  0.02018163  0.01519018  0.00709303  0.0

In [50]:
pred_words = []
pred_limit = m_cfg['max_en_pred']
for row in F.sigmoid(train_utts["probs"]).data:
    pred_inds = xp.where(row >= mean_pos_scores)[0]
    if len(pred_inds) > pred_limit:
        pred_inds = xp.argsort(row)[-pred_limit:][::-1]
    #pred_words.append([bow_dict['i2w'][i] for i in pred_inds.tolist()])
    pred_words.append([i for i in pred_inds.tolist() if i > 3])
    print(row)
    print(mean_pos_scores)
    print(pred_inds)
    break

train_prec, train_rec = basic_precision_recall(train_utts["refs"], pred_words)
print(train_prec, train_rec)

[ 0.51544869  0.40077111  0.53831834  0.57759237  0.06914282  0.13372895
  0.12780462  0.17173462  0.07306941  0.0349768   0.11242215  0.03428413
  0.19470397  0.01632826  0.07281633  0.00395539  0.0192567   0.01851128
  0.07345533  0.00187328  0.02366112  0.02286559  0.03503358  0.0583662
  0.02522211  0.08731006  0.03314728  0.15134153  0.05322033  0.08109252
  0.05765366  0.10448772  0.03016962  0.03833305  0.05536424  0.02219497
  0.04001034  0.08084626  0.00168835  0.00555753  0.01220204  0.03535946
  0.04005387  0.01693087  0.02857115  0.01161144  0.02318759  0.00666831
  0.01684841  0.01795754  0.01862131  0.04898268  0.0231012   0.01412547
  0.04483757  0.03984403  0.02511185  0.02505676  0.000625    0.0297839
  0.00900214  0.01389109  0.03104129  0.008212    0.02086379  0.01783746
  0.04733364  0.01644694  0.03997594  0.03147162  0.03201365  0.00521437
  0.01047211  0.00311684  0.00436865  0.01912993  0.0278133   0.02110873
  0.01631512  0.02018163  0.01519018  0.00709303  0.0

In [ ]:
pred_words[:5], train_utts["refs"][:5]

In [ ]:
sum([1 if (check_index in p) else 0 for p in pred_words]), len(pred_words)

### OLD

In [ ]:
utts, dev_loss = feed_model(model,
                            optimizer=optimizer,
                            m_dict=map_dict[dev_key],
                            b_dict=bucket_dict[dev_key],
                            vocab_dict=vocab_dict,
                            bow_dict=bow_dict,
                            batch_size=batch_size,
                            x_key=enc_key,
                            y_key=dec_key,
                            train=False,
                            input_path=input_path,
                            max_dec=m_cfg['max_en_pred'],
                            t_cfg=t_cfg,
                            use_y=False)

In [ ]:
PRED_THRESH = 0.1
dev_pred_words = []
pred_limit = m_cfg['max_en_pred']
for row in F.sigmoid(utts["probs"]).data:
    dev_pred_inds = xp.where(row >= PRED_THRESH)[0]
    if len(dev_pred_inds) > pred_limit:
        dev_pred_inds = xp.argsort(row)[-pred_limit:][::-1]
    #pred_words.append([bow_dict['i2w'][i] for i in pred_inds.tolist()])
    dev_pred_words.append([i for i in dev_pred_inds.tolist() if i > 3])

In [ ]:
basic_precision_recall(utts["refs"], dev_pred_words)

In [ ]:
dev_pred_words = get_pred_words_from_probs(dev_utts["probs"],
                                           0.1,
                                           m_cfg['max_en_pred'])
print(basic_precision_recall(dev_utts["refs"], dev_pred_words))
dev_pred_words = get_pred_words_from_probs(dev_utts["probs"],
                                           mean_pos_scores,
                                           m_cfg['max_en_pred'])
print(basic_precision_recall(dev_utts["refs"], dev_pred_words))

In [27]:
dev_prec, dev_rec = basic_precision_recall(dev_utts["refs"], dev_pred_words)

NameError: name 'dev_pred_words' is not defined

In [ ]:
dev_prec, dev_rec

In [ ]:
pred_words = []
pred_limit = m_cfg['max_en_pred']
for row in F.sigmoid(train_utts["probs"]).data:
    pred_inds = xp.where(row >= mean_pos_scores)[0]
    if len(pred_inds) > pred_limit:
        pred_inds = xp.argsort(row)[-pred_limit:][::-1]
    #pred_words.append([bow_dict['i2w'][i] for i in pred_inds.tolist()])
    pred_words.append([i for i in pred_inds.tolist() if i > 3])

train_prec, train_rec = basic_precision_recall(train_utts["refs"], pred_words)
print(train_prec, train_rec)

In [ ]:
dev_pred_words

In [ ]:
xp.all(utts["probs"][:10] == utts["probs"][10:20])

In [ ]:
utts["probs"].shape

In [ ]:
F.sigmoid(utts["probs"][:2,100:115])

In [ ]:
utts["probs"].T[4]

In [ ]:
utts["refs"][:5]

In [ ]:
len([p for p in utts["preds"] if len(p) > 0]), len([p for p in utts["refs"] if len(p) > 0])

In [ ]:
basic_precision_recall(utts["refs"], utts["preds"])

In [ ]:
F.binary_accuracy()

In [ ]:
basic_precision_recall([[[]], [[]], [[]], [[1]]], [[1], [], [], [1]])

In [ ]:
utts["refs"][:2], utts["preds"][:2]

In [ ]:
"".join([vocab_dict['en_c']['i2w'][i].decode() for i in pred_sents[100]])

In [ ]:
dec_key

In [ ]:
# Eval parameters
ref_index = -1
min_len, max_len= 0, 200
# min_len, max_len = 0, 1
displayN = 50
m_dict=map_dict[dev_key]
# wavs_path = os.path.join(m_cfg['data_path'], "wavs")
wavs_path = os.path.join("../chainer2/speech2text/both_fbank_out/", "wavs")
v_dict = vocab_dict[dec_key]
key = m_cfg['dev_set']

In [ ]:
max_len

In [ ]:
len(m_dict['20051009_182032_217_fsp-B-1'][dec_key])

In [ ]:
fsh_filt_pred, fsh_filt_utts = zip(*sorted([(p,u) for p, u in zip(pred_sents, utts) if (len(m_dict[u]['es_w']) >= min_len) and 
                                        (len(m_dict[u]['es_w']) <= max_len)], key=lambda t:t[1]))

In [ ]:
print("length filtered utterances = {0:d}".format(len(fsh_filt_utts)))

In [ ]:
813 / 3977, (50+115+15+171) / 3977, (50+115+15+171) / 813

In [ ]:
display_words(m_dict, v_dict, 
              fsh_filt_pred, 
              fsh_filt_utts, dec_key, 
              key, 
              play_audio=True, 
              displayN=displayN)

In [ ]:
es_ref = []
en_ref = []
for u in fsh_filt_utts:
    es_ref.append(" ".join([w.decode() for w in m_dict[u]['es_w']]))
    if type(m_dict[u][dec_key]) == list:
        en_ref.append(" ".join([w.decode() for w in m_dict[u]['en_w']]))
    else:
        en_ref.append(" ".join([w.decode() for w in m_dict[u]['en_w'][0]]))

en_pred = []
join_str = ' ' if dec_key.endswith('_w') else ''

In [ ]:
os.chdir("..")
os.chdir("/afs/inf.ed.ac.uk/group/project/lowres/work/speech2text")

In [ ]:
type(fsh_filt_pred[0])

In [ ]:
b, chrf, h, r = calc_bleu(m_dict, 
                          v_dict, 
                          fsh_filt_pred, 
                          fsh_filt_utts, 
                          dec_key, 
                          ref_index=ref_index)

print("BLEU score on: {0:s} = {1:.2f}".format(key, b * 100))
print("-"*60)

model_refs = {u: mr for u, mr in zip(fsh_filt_utts, r)}
model_hyps = {u: mh for u, mh in zip(fsh_filt_utts, h)}

all_weights=[(1.,0.,0.,0.),
             (0.,1.,0.,0.),
             (0.,0.,1.,0.),
             (0.,0.,0.,1.),
             (1./2,1./2,0.,0.),
             (1./3,1./3,1./3,0.),
             (.25,.25,.25,.25)]

In [ ]:
corpus_bleu(r, h, smoothing_function=smooth_fun.method2)

In [ ]:
en_ref, en_hyp = write_predictions_to_file(m_dict, v_dict, fsh_filt_pred, fsh_filt_utts, 
                                           dec_key, key, stemmify=False)

In [ ]:
model_refs_dict = {u:v[0].strip().split() for u, v in zip(fsh_filt_utts, en_ref)}
model_hyps_dict = {u:v.strip().split() for u, v in zip(fsh_filt_utts, en_hyp)}

In [ ]:
pickle.dump(model_refs_dict, open(os.path.join(m_cfg['model_dir'], "model_s2t_refs.dict"), "wb"))
pickle.dump(model_hyps_dict, open(os.path.join(m_cfg['model_dir'], "model_s2t_hyps.dict"), "wb"))

In [ ]:
len(en_hyp)

In [ ]:
_, _ = corpus_precision_recall(r, h)

In [ ]:
model_prec_recall = {'precision' : {}, 'recall' : {}, "tp": 0, "tc": 0, "tr": 0}

for utt_id, ref, hyp in zip(fsh_filt_utts, r, h):
    es_ref = [w.decode() for w in m_dict[utt_id]['es_w']]
    
    pval, rval = modified_precision_recall(ref, hyp, n=1)
    model_prec_recall['tc'] += pval.numerator
    model_prec_recall['tp'] += pval.denominator
    model_prec_recall['tr'] += rval.denominator

    model_prec_recall['precision'][utt_id] = float(pval)
    model_prec_recall['recall'][utt_id] = float(rval)
# end for
    
model_prec_recall['total_precision'] = model_prec_recall['tc'] / model_prec_recall['tp']
model_prec_recall['total_recall'] = model_prec_recall['tc'] / model_prec_recall['tr']

In [ ]:
print("\t\tModel metrics")
print("-"*60)
print("{0:10s} = {1:0.3f}\n{2:10s} = {3:0.3f}".format("precision",
                                                      model_prec_recall['total_precision'],
                                                      "recall",
                                                      model_prec_recall['total_recall']))


In [ ]:
print("model-s2t BLEU score:")
"{0:0.3f}".format(100.0 * corpus_bleu(model_refs.values(), model_hyps.values()))

In [ ]:
print("-"*60)
print("\t\tMODEL")
print("-"*60)
_, _ = corpus_precision_recall(model_refs.values(), model_hyps.values())

In [ ]:
print("-"*60)
print("\t\tMODEL")
print("-"*60)
metrics = {"p": {1: [], 2: [], 3: [], 4: []},
           "r": {1: [], 2: [], 3: [], 4: []}}

for ix in range(len(list(model_refs.values())[0])):
    temp_refs = [[i[ix]] for i in model_refs.values()]
    print("-"*60)
    print("\t\tUsing reference = {0:d}".format(ix+1))
    print("-"*60)
    ps, rs = corpus_precision_recall(temp_refs, model_hyps.values())
    for i, (p, r) in enumerate(zip(ps, rs)):
        metrics['p'][i+1].append(p)
        metrics['r'][i+1].append(r)
    

In [ ]:
def get_mean_std(vals_dict):
#     print("estimated mean")
    k = []
    u = []
    s = []
    for m, vals in vals_dict.items():
        k.append(m)
        u.append(np.mean(vals))
        s.append(np.std(vals)/np.sqrt(len(vals)))
        print("{0:10d}-gram = {1:.2f} ± {2:.2f}".format(m, np.mean(vals), np.std(vals)/np.sqrt(len(vals))))
        
    print(",".join(map(lambda x : "{0:.2f}".format(x), u)))
    print(",".join(map(lambda x : "{0:.2f}".format(x), s)))
    return k, u, s

In [ ]:
s2w_p_r = {}

print("Precision:")
_, s2w_p_r['p'], s2w_p_r['p_std'] = get_mean_std(metrics['p'])

print("Recall:")
_, s2w_p_r['r'], s2w_p_r['r_std'] = get_mean_std(metrics['r'])

In [ ]:
with open(os.path.join(m_cfg['model_dir'], "s2w_p_r.json"), "w") as out_f:
    json.dump(s2w_p_r, out_f, indent=4)

In [ ]:
model_predictions = {}
for u, hyp in zip(fsh_filt_utts, h):
    model_predictions[u] = hyp

### word level analysis

In [ ]:
top_k = 100
min_word_len = 1

In [ ]:
stop_words = set(nltk.corpus.stopwords.words("english"))
len(stop_words)

In [ ]:
def get_words(m_dict):
    words = []
    for u in m_dict:
        if type(m_dict[u]['en_w']) == list:
            words.extend([w.decode() for w in m_dict[u]['en_w']])
        else:
            for ref in m_dict[u]['en_w']:
                words.extend([w.decode() for w in ref])
    return Counter(words)

In [ ]:
# words in train
train_words = get_words(map_dict['fisher_train'])
train_words_top_k = [(w,f) for w, f in sorted(train_words.items(), reverse=True, key=lambda t:t[1]) 
                     if w not in stop_words and len(w) >= min_word_len][:top_k]

train_only_words = set(train_words.keys())

print("{0:20s} | {1:10d}".format("# train word types", len(train_words)))
print("{0:20s} | {1:10d}".format("# train word tokens", sum(train_words.values())))

In [ ]:
train_words_top_k[:5]

In [ ]:
[(w,f) for w,f in train_words_top_k if "'" in w]

In [ ]:
dev_words = get_words(map_dict['fisher_dev'])
dev_words_top_k = [(w,f) for w, f in sorted(dev_words.items(), reverse=True, key=lambda t:t[1]) 
                     if w not in stop_words and len(w) >= min_word_len][:top_k]

dev_only_words = set(dev_words.keys())

In [ ]:
dev_words_top_k[:5]

In [ ]:
oov_words = {w:f for w,f in dev_words.items() if w not in train_only_words}

In [ ]:
print("{0:20s} | {1:10d}".format("# dev word types", len(dev_only_words)))
print("{0:20s} | {1:10d}".format("# dev word tokens", sum(dev_words.values())))

print("{0:20s} | {1:10d}".format("# oov word types", len(oov_words)))
print("{0:20s} | {1:10d}".format("# oov word tokens", sum(oov_words.values())))


In [ ]:
"{0:.1f}%".format(sum(oov_words.values()) / sum(dev_words.values()) * 100)

Look at utterances which our model is doing better at, and compare to Google.

Are we doing better on a few calls only? Is there any particular speaker or call messing up our results?

### Query task

In [ ]:
crises_lex_fname = "../criseslex/CrisisLexLexicon/CrisisLexRec.txt"
train_text_fname= "../installs/fisher-callhome-corpus/corpus/ldc/fisher_train.en"
topics_fname = "../criseslex/fsp06_topics_in_english.txt"

In [ ]:
topics = [ "peace", "Music", "Marriage", "Religion", "Cell phones", 
           "Dating", "Telemarketing and SPAM", "Politics", "Travel", 
           "Technical devices", "Healthcare", "Advertisements", "Power", 
           "Occupations", "Movies", "Welfare", "Breaking up", "Location", 
           "Justice", "Memories", "Crime", "Violence against women", "Equality", 
            "Housing", "Immigration",     
            # new topics
           "Interracial", "Christians", "muslims", "jews", "e-mail", 
           "phone", "democracy", "Democratic", "Republican", "technology", 
           "leadership", "community", "jury", "police", "inequality", 
           "renting", "Violence", "immigrants", "immigrant", "skilled", 
           "Telemarketing", "SPAM", "skill", "job", "health", "mobile", 
            "ads", "physical", "emotional", "bubble", "rent", "economy", 
            "abuse", "women", "city", "country", "suburban", "dollar", 
            "united states", "laws", "phone", "race", "biracial", "interracial", 
            "marriage", "lyrics", "sexuality", "medicine", "television", "european",
            "home", "protect", "spouse", "language", "cellphone", "money",
            "doctor", "insurance", "cigarettes", "alcohol", "income", "salary",
            "class", "censor", "rating", "programs", "government",
            "relationship", "legal", "event", "life", "safe", "victim", "cops",
            "wage", "illegal"
            ]
topics = list(set(t.lower() for t in topics))
topics_stem = [stem(t) for t in topics]

In [ ]:
dev_words_stem = {}
for w in dev_words:
    stem_w = stem(w)
    if stem_w not in dev_words_stem:
        dev_words_stem[stem_w] = 0
    dev_words_stem[stem_w] += dev_words[w]
# end for

In [ ]:
len(topics)

In [ ]:
# NOTE this checks for # of occurrences of the word in the specified text
# We later use the # of utterrances, which will be lower or equal to this number
prune_topics = [t for t in topics if dev_words_stem.get(stem(t),0) > 10 and dev_words_stem.get(stem(t),0) < 100]
len(prune_topics)

In [ ]:
prune_topics[:5]

In [ ]:
random.seed("selec_query_terms")
sel_topics = random.sample(prune_topics, min(len(prune_topics), 100))

In [ ]:
len(sel_topics)

In [ ]:
def find_all_terms_references(utt_text, terms, text_key="en_w"):
    terms_set = set([stem(i) for i in terms])
    terms_search_res = {}
    full_words = {}

    for u in tqdm(utt_text, ncols=80):
        l = (utt_text[u][text_key] if type(utt_text[u]) == dict 
             else utt_text[u])
        for r in l:
            for w in set(r):
                decoded_w = w.decode() if type(w) != str else w
                w_to_search = stem(decoded_w)
                if w_to_search in terms_set:
                    if w_to_search not in terms_search_res:
                        terms_search_res[w_to_search] = set()
                        full_words[w_to_search] = set()
                    terms_search_res[w_to_search].add(u)
                    full_words[w_to_search].add(decoded_w)
                # end if found
            # end for current reference
        # end for all references
    # end for all utterances
    return terms_search_res, full_words


def find_all_terms_predictions(utt_text, terms):
    terms_set = set([stem(i) for i in terms])
    terms_search_res = {}
    full_words = {}

    for u in tqdm(utt_text, ncols=80):
        r = utt_text[u]
        for w in set(r):
            w_to_search = stem(w)
            if w_to_search in terms_set:
                if w_to_search not in terms_search_res:
                    terms_search_res[w_to_search] = set()
                    full_words[w_to_search] = set()
                terms_search_res[w_to_search].add(u)
                full_words[w_to_search].add(u)
                # end if found
            # end for current reference
    # end for all utterances
    return terms_search_res, full_words

In [ ]:
def terms_prec_recall(preds, refs):
    prec_recall = {}
    prec_recall = {'t':0, 'tp':0, 'tc':0, 'terms':{}}
    for term in refs.keys():
        prec_recall['terms'][term] = {}
        prec_recall['terms'][term]['t'] = len(refs[term])
        preds_occ = preds.get(term, set())
        prec_recall['terms'][term]['tp'] = len(preds_occ)
        prec_recall['terms'][term]['tc'] = len(refs[term] & preds_occ)
        prec_recall['t'] += prec_recall['terms'][term]['t']
        prec_recall['tp'] += prec_recall['terms'][term]['tp']
        prec_recall['tc'] += prec_recall['terms'][term]['tc']
    # end for each term
    return prec_recall

In [ ]:
ref_topics, ref_topic_labels = find_all_terms_references(map_dict['fisher_dev'], sel_topics)
pred_topics, pred_topics_topics_labels = find_all_terms_predictions(model_predictions, sel_topics)

In [ ]:
len(ref_topics), len(pred_topics)

In [ ]:
ref_topics

In [ ]:
prune_topics_in_utts = {k:v for k, v in ref_topics.items() if len(v) >=3}

In [ ]:
len(prune_topics_in_utts)

In [ ]:
from  matplotlib.ticker import FuncFormatter

In [ ]:
sns.set_style("darkgrid")

In [ ]:
sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(6,10),nrows=1, ncols=1)


sorted_counts = [t[0] for t in sorted(ref_topics.items(), reverse=True, key=lambda t:len(t[1]))]
topic_count = [len(ref_topics[t]) for t in sorted_counts]

topic_labels = ["-".join(list(ref_topic_labels[t])[:3]) for t in sorted_counts]

ax = sns.barplot(x=topic_count, 
                 y=topic_labels, 
                 color=tableau20[6], alpha=.7)
                 #**{"label":"topic counts", "alpha":0.5}, ax=ax)

# ax.set_xlabel("# of utts in which topic occurs", size=20)

max_count_val = max([len(v) for v in ref_topics.values()])
min_count_val = max([len(v) for v in ref_topics.values()])
plt.xticks(list(range(0,max_count_val, 5))+[min_count_val, max_count_val])

# ax.legend(loc='upper right', bbox_to_anchor=(1.1, 0.9),
#                       ncol=1, fancybox=True, shadow=True, fontsize=20)

sns.despine(left=True, bottom=True, top=True, right=False)

for t in ax.get_yticklabels():
    t.set_fontsize(15) 
    
for i, t in enumerate(ax.get_xticklabels()):
    if i > 0 and i < (len(ax.get_xticklabels())-1):
        t.set_visible(False)
    else:
        t.set_fontsize(16)

fig.tight_layout()


fig.savefig("../criseslex/sel_topics_new.png", dpi=300)

In [ ]:
model_topics_p_r = terms_prec_recall(preds = pred_topics, refs = ref_topics)

In [ ]:
model_topics_p_r['precision'] = model_topics_p_r['tc'] / model_topics_p_r['tp']
model_topics_p_r['recall'] = model_topics_p_r['tc'] / model_topics_p_r['t']

In [ ]:
print("-"*40)
print("-------------- Query by Text")
print("-"*40)
print("{0:20s} | {1:5.2f}%".format("precision", model_topics_p_r['precision']*100.0))
print("{0:20s} | {1:5.2f}%".format("recall", model_topics_p_r['recall']*100.0))

In [ ]:
with open(os.path.join(m_cfg['model_dir'], "topics_p_r.json"), "w") as out_f:
    json.dump(model_topics_p_r, out_f, indent=4)

In [ ]:
def find_all_terms_es_references(utt_text, term):
    terms_search_res = set()
    total_word_count = 0

    for u in tqdm(utt_text, ncols=80):
        l = (utt_text[u]["es_w"] if type(utt_text[u]) == dict 
             else utt_text[u])
        #if len(l) == 1 and l[0].decode() == term:
        if set(l) == set(term.split()):
            terms_search_res.add(u)
            total_word_count += 1
            # end if found
        # end for current reference
        # end for all references
    # end for all utterances
    return terms_search_res, total_word_count


In [ ]:
set_key = "fisher_train"

In [ ]:
hmm, tot_count = find_all_terms_es_references(map_dict[set_key], b'claro')

In [ ]:
len(hmm)

In [ ]:
all_ref_words = {}
all_match_count = 0
no_match_count = 0

utt_refs_set = {}
for utt in hmm:
    #print("-"*60)
    #print(utt)
    curr_ref = {k : "" for k in range(4)}
    if "train" in set_key:
        r = map_dict[set_key][utt]["en_w"]
        en_ref_words = " ".join(w.decode() for w in r)
        curr_ref[0] = en_ref_words
        if en_ref_words not in all_ref_words:
            all_ref_words[en_ref_words] = 0
        all_ref_words[en_ref_words] += 1
    else:
        for i, r in enumerate(map_dict[set_key][utt]["en_w"]):
            en_ref_words = " ".join(w.decode() for w in r)
            curr_ref[i] = en_ref_words
            #print(" ".join(w.decode() for w in r), end="----")
            if en_ref_words not in all_ref_words:
                all_ref_words[en_ref_words] = 0
            all_ref_words[en_ref_words] += 1
    #print(curr_ref.values())

    
    utt_refs_set[utt] = set(curr_ref.values())
        

In [ ]:
len(utt_refs_set['20051023_232057_325_fsp-B-77']) if "dev" in set_key else 0

In [ ]:
 if "dev" in set_key:
    start_set = utt_refs_set['20051023_232057_325_fsp-B-77']
    for utt in utt_refs_set:
        start_set = start_set & utt_refs_set[utt]

In [ ]:
start_set

In [ ]:
start_set

In [ ]:
all_match_count = sum([len(utt_refs_set[utt]) == 1 for utt in utt_refs_set])
all_dont_match_count = sum([len(utt_refs_set[utt]) == 4  for utt in utt_refs_set])

In [ ]:
all_match_count, all_dont_match_count

In [ ]:
all_ref_words, len(all_ref_words)

In [ ]:
tot_count

In [ ]:
utt_refs_set

In [ ]:
m_dict = map_dict[set_key]

In [ ]:
input_path = os.path.join(m_cfg['data_path'], m_cfg['train_set'])

In [ ]:
sel_utts = [utt for utt in utt_refs_set]

In [ ]:
batch_data = get_batch(m_dict, enc_key, dec_key, sel_utts[:101], vocab_dict, 200, 10, input_path=input_path)

In [ ]:
batch_data['X'].shape

In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [ ]:
A = xp.asnumpy(batch_data['X'].data)

In [ ]:
type(A)

In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

distance, path = fastdtw(A[1], A[2], dist=euclidean)
print(distance)

In [ ]:
dtw_out = np.zeros((len(A), len(A)))

In [ ]:
for i in tqdm(range(len(A))):
    for j in range(i,len(A)):
        dtw_out[i,j], _ = fastdtw(A[i], A[j], dist=euclidean)

In [ ]:
dist_matrix = dtw_out + dtw_out.T - - np.diag(np.diag(dtw_out))

In [ ]:
np.mean(dist_matrix), np.std(dist_matrix)

In [ ]:
sns.heatmap(dist_matrix, cmap="YlGnBu")

In [ ]:
mask = np.zeros_like(dist_matrix)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    ax = sns.heatmap(dist_matrix, mask=mask, square=True, cmap="YlGnBu")

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

In [ ]:
X = dist_matrix

In [ ]:
import time as time
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.cluster import AgglomerativeClustering

In [ ]:
st = time.time()
ward = AgglomerativeClustering(n_clusters=6, linkage='ward').fit(X)
elapsed_time = time.time() - st
label = ward.labels_
print("Elapsed time: %.2fs" % elapsed_time)
print("Number of points: %i" % label.size)

In [ ]:
B = A.reshape(len(A),-1)

In [ ]:
B.shape

In [ ]:
pca = PCA(n_components=2)
B_r = pca.fit(B).transform(B)

In [ ]:
# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))


In [ ]:
# #############################################################################
# Plot result
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
for l in np.unique(label):
    ax.scatter(X[label == l, 0], X[label == l, 1], X[label == l, 2],
               color=plt.cm.jet(np.float(l) / np.max(label + 1)),
               s=20, edgecolor='k')
plt.title('Without connectivity constraints (time %.2fs)' % elapsed_time)


# #############################################################################
# Define the structure A of the data. Here a 10 nearest neighbors
from sklearn.neighbors import kneighbors_graph
connectivity = kneighbors_graph(X, n_neighbors=25, include_self=False)

# #############################################################################
# Compute clustering
print("Compute structured hierarchical clustering...")
st = time.time()
ward = AgglomerativeClustering(n_clusters=10, connectivity=connectivity,
                               linkage='ward').fit(X)
elapsed_time = time.time() - st
label = ward.labels_
print("Elapsed time: %.2fs" % elapsed_time)
print("Number of points: %i" % label.size)

# #############################################################################
# Plot result
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
for l in np.unique(label):
    ax.scatter(X[label == l, 0], X[label == l, 1], X[label == l, 2],
               color=plt.cm.jet(float(l) / np.max(label + 1)),
               s=20, edgecolor='k')
plt.title('With connectivity constraints (time %.2fs)' % elapsed_time)

plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
ward = AgglomerativeClustering(n_clusters=5, connectivity=connectivity,
                               linkage='ward').fit(X)

labels = ward.labels_

fig, ax = plt.subplots(figsize=(10,10))

for color, l in zip(tableau20, np.unique(labels)):
    ax.scatter(B_r[labels == l, 0], B_r[labels == l, 1], s=100,
               color=plt.cm.jet(float(l) / np.max(labels + 1)), edgecolor='k')
plt.yticks(visible=False)
plt.xticks(visible=False)
fig.savefig("agglo_dtw_claro.pdf", dpi=300)

In [ ]:
est = KMeans(n_clusters=5)
est.fit(B)
labels = est.labels_

fig, ax = plt.subplots(figsize=(10,10))

for color, l in zip(tableau20, np.unique(labels)):
    ax.scatter(B_r[labels == l, 0], B_r[labels == l, 1], s=100,
               color=plt.cm.jet(float(l) / np.max(labels + 1)), edgecolor='k')
# done
plt.yticks(visible=False)
plt.xticks(visible=False)
fig.savefig("k_means_claro.pdf", dpi=300)

In [ ]:
Counter(list(ward.labels_))

In [ ]:
label = ward.labels_

In [ ]:
np.unique(label)

In [ ]:
label

In [ ]:
for l in np.unique(label):
    ax.scatter(X[label == l, 0], X[label == l, 1], X[label == l, 2],

In [ ]:

X[label == l, 0], X[label == l, 1], X[label == l, 2]

In [ ]:
# Plot the results on an image
plt.figure(figsize=(5, 5))
plt.imshow(X, cmap=plt.cm.gray)
for l in range(n_clusters):
    plt.contour(label == l, contours=1,
                colors=[plt.cm.spectral(l / float(n_clusters)), ])
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
similarities = cosine_similarity(A)
print('pairwise dense output:\n {}\n'.format(similarities))

In [ ]:
"20051021_222225_307_fsp-B-109" in map_dict['fisher_train']

In [ ]:
def get_batch(m_dict, x_key, y_key, utt_list, vocab_dict,
              max_enc, max_dec, input_path=''):
    batch_data = {'X':[], 'y':[]}
    # -------------------------------------------------------------------------
    # loop through each utterance in utt list
    # -------------------------------------------------------------------------
    for u in utt_list:
        # ---------------------------------------------------------------------
        #  add X data
        # ---------------------------------------------------------------------
        if x_key == 'sp':
            # -----------------------------------------------------------------
            # for speech data
            # -----------------------------------------------------------------
            # get path to speech file
            utt_sp_path = os.path.join(input_path, "{0:s}.npy".format(u))
            if not os.path.exists(utt_sp_path):
                # for training data, there are sub-folders
                utt_sp_path = os.path.join(input_path,
                                           u.split('_',1)[0],
                                           "{0:s}.npy".format(u))
            if os.path.exists(utt_sp_path):
                x_data = xp.load(utt_sp_path)
                # truncate max length
                batch_data['X'].append(x_data[:max_enc])
            else:
                # -------------------------------------------------------------
                # exception if file not found
                # -------------------------------------------------------------
                print("ERROR!! file not found: {0:s}".format(utt_sp_path))
                # -------------------------------------------------------------
        else:
            # -----------------------------------------------------------------
            # for text data
            # -----------------------------------------------------------------
            x_ids = [vocab_dict[x_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][x_key]]
            x_ids = xp.asarray(x_ids, dtype=xp.int32)
            batch_data['X'].append(x_ids[:max_enc])
            # -----------------------------------------------------------------
        # ---------------------------------------------------------------------
        #  add labels
        # ---------------------------------------------------------------------
        if type(m_dict[u][y_key]) == list:
            en_ids = [vocab_dict[y_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key]]
        else:
            # dev and test data have multiple translations
            # choose the first one for computing perplexity
            en_ids = [vocab_dict[y_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key][0]]
        y_ids = [GO_ID] + en_ids[:max_dec-2] + [EOS_ID]
        batch_data['y'].append(xp.asarray(y_ids, dtype=xp.int32))
        # ---------------------------------------------------------------------
    # -------------------------------------------------------------------------
    # end for all utterances in batch
    # -------------------------------------------------------------------------
    if len(batch_data['X']) > 0 and len(batch_data['y']) > 0:
        batch_data['X'] = F.pad_sequence(batch_data['X'], padding=PAD_ID)
        batch_data['y'] = F.pad_sequence(batch_data['y'], padding=PAD_ID)
    return batch_data

In [ ]:
# puerto rico utterance
# "i 'm from puerto rico but i live here in denver colorado"
" ".join(w.decode() for w in map_dict['fisher_dev']['20051023_232057_325_fsp-A-3']['en_w'][0])